In [20]:
import sys
sys.path.insert(0, '..')

import sympy as sy
from src.dynamical_system import DynamicalSystem
from src.componentwise_expression import ComponentwiseExpression
from src.util import *

In [12]:
A,B,C,D,E,F,N = sy.symbols('A B C D E F N', real = True)
a,b,c,alpha,beta = sy.symbols('a b c alpha beta', real = True)

x,y,z = sy.symbols('x y z', real = True)

equation = A + B*x + C*x**2

In [13]:
solution = find_solution_of_equation_by_inserting_values(equation, x, {A: 1, B: 0, C: 2}, show_process=True)

A=-B - C
B=-C
C=0
remaining terms of equation:
0


In [14]:
equation = (A*sy.cos(x) + B*sy.sin(x) + C)*(1+b*sy.cos(x-beta)) + (D*sy.cos(x) + E*sy.sin(x) + F)*(1+a*sy.cos(x-alpha)) - N

In [15]:
temporary_coefficients = get_coefficients_of_trigonometric_expression(equation, x, 2)

for c in temporary_coefficients:
    print(c)

A*b*cos(beta)/4 - B*b*sin(beta)/4 + D*a*cos(alpha)/4 - E*a*sin(alpha)/4
-A*b*sin(beta)/4 - B*b*cos(beta)/4 - D*a*sin(alpha)/4 - E*a*cos(alpha)/4
A/2 + C*b*cos(beta)/2 + D/2 + F*a*cos(alpha)/2
-B/2 - C*b*sin(beta)/2 - E/2 - F*a*sin(alpha)/2
A*b*cos(beta)/2 + B*b*sin(beta)/2 + C + D*a*cos(alpha)/2 + E*a*sin(alpha)/2 + F - N


In [16]:
equation = A*x**2 + B*x + C + D*x*y + E*x*y + D*x**2 + D + 4*z**4*x

print(equation)

expr = ComponentwiseExpression(equation)
print(expr._expression)

expr.split(x)

print(expr._expression)

expr.split(y)

print(expr._expression)

expr.split(z)

print(expr._expression)

expr.prune()

print(expr._expression)

sy.cancel(expr.sum_up() - equation)

expr.show(number_of_ops=0)

A*x**2 + B*x + C + D*x**2 + D*x*y + D + E*x*y + 4*x*z**4
{1: A*x**2 + B*x + C + D*x**2 + D*x*y + D + E*x*y + 4*x*z**4}
{x**2: A + D, x: B + D*y + E*y + 4*z**4, 1: C + D}
{x**2: A + D, x*y: D + E, x: B + 4*z**4, 1: C + D}
{x**2: A + D, x*y: D + E, x*z**4: 4, x*z**3: 0, x*z**2: 0, x*z: 0, x: B, 1: C + D}
{x**2: A + D, x*y: D + E, x*z**4: 4, x: B, 1: C + D}
x*z**4: 4
x: B


In [17]:
system = DynamicalSystem("lorenz")
system.show()

<IPython.core.display.Math object>

In [18]:
x,y,z = system._variables
beta, sigma, rho = system._parameters

In [ ]:
def get_multivariate_polynomial(variables: list[sy.Symbol], maximum_power: int) -> sy.Expr:
    polynomial = 0
    for power in range(maximum_power+1):
        for var in variables:
            polynomial += var**power
    return polynomial

### reproducing polynomial Koopman eigenfunctions

In [19]:
a,b,c,ld = sy.symbols('a b c lambda', real = True)

observable = a*x**2 + b*y**2 + c*z**2

remainder = ComponentwiseExpression(system.get_time_derivative_of_observable(observable) - ld*observable)

for var in [x,y,z]:
    remainder.split(var)
    remainder.prune()

#remainder.subs({b: c, a: 0, rho: 0, ld: -2, beta: 1})
remainder.subs({b: c, ld: -2*sigma, c: 0, sigma: 0})
#remainder.subs({b:c, ld: -2*sigma, sigma: 1, beta:1, a:-c*rho})

remainder.prune()

remainder.show()

### polynomial + exponential

In [342]:
#def get_polynomial_expression(var):

In [379]:
a, b, c, d, e, f, g, h ,i, j, k, l, m, n, o, ld = sy.symbols('a b c d e f g h i j k l m n o lambda', real = True)
A, B, C, D, E, F, G, H, I, J, K, L, M, N, O = sy.symbols('A B C D E F G H I J K L M N O', real = True)

exp_polynomial = A*x + B*y + C*z + D*x**2 + E*y**2 + F*z**2 + G*x*y + H*x*z + I*y*z + J*x**3 + K*y**3 + L*z**3 + M*x**2*z + N*x*y*z + O*y**2*z
src_polynomial = a*x + b*y + c*z + d*x**2 + e*y**2 + f*z**2 + g*x*y + h*x*z + i*y*z + j*x**3 + k*y**3 + l*z**3 + m*x**2*z + n*x*y*z + o*y**2*z
observable = src_polynomial*sy.exp(exp_polynomial)

remainder = componentwise_expression((system.get_time_derivative_of_observable(observable) - ld*observable)*sy.exp(-exp_polynomial))

for var in [x,y,z]:
    remainder.split(var)
    remainder.prune()

#solution = {a: 0, H: 0, G: 0, b: c, I:0, B:0, E:F, F:0, rho:0, ld: -2, beta:1, C:0, sigma:0}

# trivial solution: \dot{x} = 0
# solution = {H: 0, G: 0, I: 0, F:0, E: 0, C:0, B:0, f:0, i: 0, h: 0, e: 0, c:0, g:0, b:0, sigma:0, ld:0}

# Kus 1983 solutions
#solution = {H:0, I:0, i:0, N:0, M:0, G:0, B:0, D:0, A:0, E:0, C:0, h:0, b:0, a:0, F:0, m: -4*n*sigma, e: f - 4*n*sigma**2,
#            g: 4*beta*n*sigma + 4*ld*n*sigma + 8*n*sigma**2, c:0, f:0}

#solution = {H:0, I:0, F:0, f:0, C:0, i:0, ld:-2*sigma, h: 0, b:0, a:0, B:0, n:0, N:0, M:0, E:0, G:0, D:0, m:0, A:0, g:0, e:0, c:-2*d*sigma, beta:2*sigma}

solution = {M:0, N:0, O:0, L:0, l:0, I:0, K:0, G:0, k:0, e:0, F:0, E:0, o:0, H:0, J:0, B:0, n:0, f:0, C:0, i:0, D:0, m:0, j:0,d:0, g:0, h:0, a:0, b:0, c:0}

remainder.subs(solution)
remainder.prune()

remainder.show(number_of_ops=3)

In [380]:
observable.subs(solution)

0

### rational 

In [ ]:
a, b, c, d, e, f, g, h ,i, m, n, ld = sy.symbols('a b c d e f g h i m n lambda', real = True)
A, B, C, D, E, F, G, H, I, M, N, = sy.symbols('A B C D E F G H I M N', real = True)

denominator = A*x + B*y + C*z + D*x**2 + E*y**2 + F*z**2 + G*x*y + H*x*z + I*y*z + M*x**2*z + N*x**4
numerator = a*x + b*y + c*z + d*x**2 + e*y**2 + f*z**2 + g*x*y + h*x*z + i*y*z + m*x**2*z + n*x**4

observable = numerator/denominator

remainder = componentwise_expression((system.get_time_derivative_of_observable(numerator)*denominator - system.get_time_derivative_of_observable(denominator)*numerator - ld*denominator*numerator))

for var in [x,y,z]:
    remainder.split(var)
    remainder.prune()

#solution = {a: 0, H: 0, G: 0, b: c, I:0, B:0, E:F, F:0, rho:0, ld: -2, beta:1, C:0, sigma:0}

# trivial solution: \dot{x} = 0
# solution = {H: 0, G: 0, I: 0, F:0, E: 0, C:0, B:0, f:0, i: 0, h: 0, e: 0, c:0, g:0, b:0, sigma:0, ld:0}

solution = {N: 0, n:0, F: 0, f: 0, C:0, c:0, m:0, M:0} #N:0, M:0, H:0, G:0, I:0, B:0, i:0, h:0, F:0, b:0, a:0, c:0, f:0, A:0, E: 0}
#solution = {H:0, I:0, i:0, N:0, M:0, G:0, B:0, D:0, A:0, E:0, C:0, h:0, b:0, a:0, F:0, m: -4*n*sigma, e: f - 4*n*sigma**2,
#            g: 4*beta*n*sigma + 4*ld*n*sigma + 8*n*sigma**2, c:0, f:0}

remainder.subs(solution)
remainder.prune()

remainder.show(number_of_ops=3)

x**4*y: D*h - H*d
x**4*z: -D*g + G*d
x**2*y**2*z: E*g - G*e
x**2*z**3: -H*i + I*h
x*y**4: E*i - I*e
x*y**2*z**2: E*i - I*e


In [385]:
observable.subs(solution)

(a*x + b*y + d*x**2 + e*y**2 + g*x*y + h*x*z + i*y*z)/(A*x + B*y + D*x**2 + E*y**2 + G*x*y + H*x*z + I*y*z + J*x**3 + K*y**3 + L*z**3 + O*y**2*z)

# van der Pol

In [4]:
system = DynamicalSystem("van_der_Pol")
system.show()

<IPython.core.display.Math object>

In [104]:
x,y = system._variables
eps, = system._parameters

In [79]:
a, b, c, d, e, f, g, h ,i, j, ld = sy.symbols('a b c d e f g h i j lambda', real = True)
A, B, C, D, E, F, G, H, I, J, = sy.symbols('A B C D E F G H I J', real = True)

poly_1 = A + B*x + C*y + D*x**2 + E*x*y + F*y**2 + G*x**3 + H*x**2*y + I*x*y**2 + J*y**3
poly_2 = a + b*x + c*y + d*x**2 + e*x*y + f*y**2 + g*x**3 + h*x**2*y + i*x*y**2 + j*y**3

In [123]:
observable, remainder = get_remainder_with_exponential_ansatz(system, poly_1, poly_2, ld)

for var in [x,y]:
    remainder.split(var)
    remainder.prune()

solution = {A: 0, C: -B*ld, E: B*c, H: B*e, G:0, h:0, J:0, e:0, D:0, i:0, j:0, f:0, I: F*(b + c*eps), F: -B*b/2 - B*c*ld, g:0, b:-c*eps, c:0}

remainder.subs(solution)
remainder.prune()

remainder.show() #number_of_ops=10)

x**2*y: -2*B*d + B*epsilon*lambda
x*y**2: 2*B*d*lambda
y: -B*epsilon*lambda + B*lambda**2 + B


In [124]:
observable.subs(solution)

(-B*lambda*y + B*x)*exp(a + d*x**2)